In [2]:
# ==== Step 1. Import Libraries ====
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import MinMaxScaler
import pickle, joblib, warnings, os
from prophet import Prophet
import xgboost as xgb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping
from statsmodels.tsa.arima.model import ARIMA
from math import sqrt
import zipfile # Import zipfile

warnings.filterwarnings("ignore")
plt.style.use('seaborn-v0_8-whitegrid')

# ==== Step 0. Load Dataset ====
# Unzip the file first
zip_path = "/content/city_day.csv.zip"
extract_path = "/content/"
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Load the extracted CSV
csv_path = os.path.join(extract_path, "city_day.csv")
df = pd.read_csv(csv_path)
print("Raw dataset shape:", df.shape)

# ==== Step 2. Data Cleaning ====
df = df[['Date', 'City', 'PM2.5']].dropna()
df['Date'] = pd.to_datetime(df['Date'])
df = df[df['PM2.5'] > 0]
df = df.sort_values(['City', 'Date'])

# Create results folder
os.makedirs("results", exist_ok=True)
clean_path = "results/clean_city_data.csv"
df.to_csv(clean_path, index=False)
print(f"✅ Clean dataset saved at: {os.path.abspath(clean_path)}")

# ==== Step 3. Loop Over Each City ====
city_results = []

for city in df['City'].unique():
    print(f"\n================= {city} =================")

    city_df = df[df['City'] == city].copy()
    city_df.set_index('Date', inplace=True)
    series = city_df['PM2.5'].asfreq('D').fillna(method='ffill')

    # Skip very small datasets
    if len(series) < 100:
        print(f"Skipping {city} (too few records)")
        continue

    # Split data
    train_size = int(len(series)*0.8)
    train, test = series[:train_size], series[train_size:]

    # Ensure test set has at least 10 data points
    if len(test) < 10:
        print(f"Adjusting split for {city} to ensure test set has at least 10 records.")
        test_size = min(10, len(series)) # Ensure test_size is not more than total series length
        train_size = len(series) - test_size
        train, test = series[:train_size], series[train_size:]
        if len(test) == 0:
             print(f"Skipping {city} (still too few records after adjustment)")
             continue


    results = {}

    # --- ARIMA ---
    try:
        if len(test) > 0:
            model_arima = ARIMA(train, order=(1,1,1)).fit()
            pred_arima = model_arima.forecast(len(test))
            mae_arima = mean_absolute_error(test, pred_arima)
            rmse_arima = sqrt(mean_squared_error(test, pred_arima))
            results['ARIMA'] = (mae_arima, rmse_arima)
        else:
            print(f"ARIMA skipped for {city}: test set is empty.")
    except Exception as e:
        print(f"ARIMA failed for {city}: {e}")

    # --- Prophet ---
    try:
        if len(test) > 0:
            prophet_df = train.reset_index()
            prophet_df.columns = ['ds', 'y']
            model_prophet = Prophet()
            model_prophet.fit(prophet_df)
            future = pd.DataFrame(test.index, columns=['ds'])
            forecast = model_prophet.predict(future)
            pred_prophet = forecast['yhat'].values
            mae_prophet = mean_absolute_error(test, pred_prophet)
            rmse_prophet = sqrt(mean_squared_error(test, pred_prophet))
            results['Prophet'] = (mae_prophet, rmse_prophet)
        else:
            print(f"Prophet skipped for {city}: test set is empty.")
    except Exception as e:
        print(f"Prophet failed for {city}: {e}")

    # --- XGBoost ---
    try:
        lags = 5
        X, y = [], []
        for i in range(lags, len(train)):
            X.append(train.values[i-lags:i])
            y.append(train.values[i])
        X_train = np.array(X)
        y_train = np.array(y)

        X_test, y_test = [], []
        for i in range(lags, len(test)):
            X_test.append(test.values[i-lags:i])
            y_test.append(test.values[i])
        X_test = np.array(X_test)
        y_test = np.array(y_test)

        # Ensure there are enough samples for XGBoost after creating lagged features
        if len(X_train) > 0 and len(X_test) > 0:
            scaler = MinMaxScaler()
            X_train_scaled = scaler.fit_transform(X_train)
            X_test_scaled = scaler.transform(X_test)

            model_xgb = xgb.XGBRegressor(n_estimators=200, learning_rate=0.1)
            model_xgb.fit(X_train_scaled, y_train)
            pred_xgb = model_xgb.predict(X_test_scaled)
            mae_xgb = mean_absolute_error(y_test, pred_xgb)
            rmse_xgb = sqrt(mean_squared_error(y_test, pred_xgb))
            results['XGBoost'] = (mae_xgb, rmse_xgb)
        else:
             print(f"XGBoost skipped for {city}: not enough data for lagged features.")

    except Exception as e:
        print(f"XGBoost failed for {city}: {e}")

    # --- LSTM ---
    try:
        # Ensure there are enough samples for LSTM after creating lagged features
        if len(X_train) > 0 and len(X_test) > 0:
            X_train_lstm = X_train_scaled.reshape((X_train_scaled.shape[0], X_train_scaled.shape[1], 1))
            X_test_lstm = X_test_scaled.reshape((X_test_scaled.shape[0], X_test_scaled.shape[1], 1))

            model_lstm = Sequential([
                LSTM(64, activation='relu', input_shape=(lags,1)),
                Dense(1)
            ])
            model_lstm.compile(optimizer='adam', loss='mse')
            model_lstm.fit(X_train_lstm, y_train, epochs=20, batch_size=16,
                           validation_split=0.2, verbose=0,
                           callbacks=[EarlyStopping(patience=3, restore_best_weights=True)])
            pred_lstm = model_lstm.predict(X_test_lstm).flatten()
            mae_lstm = mean_absolute_error(y_test, pred_lstm)
            rmse_lstm = sqrt(mean_squared_error(test.values[lags:], pred_lstm)) # Ensure test values match pred_lstm length
            results['LSTM'] = (mae_lstm, rmse_lstm)
        else:
            print(f"LSTM skipped for {city}: not enough data for lagged features.")

    except Exception as e:
        print(f"LSTM failed for {city}: {e}")


    # ==== Step 4. Evaluate & Save Best ====
    if not results:
        continue

    result_df = pd.DataFrame(results, index=['MAE','RMSE']).T
    best_model_name = result_df['RMSE'].idxmin()
    best_rmse = result_df.loc[best_model_name, 'RMSE']
    print(f"\n📊 {city} results:\n", result_df)
    print(f"🏆 Best model for {city}: {best_model_name} (RMSE={best_rmse:.2f})")

    # ==== Step 5. Save Best Model ====
    if best_model_name == 'ARIMA':
        path = f"results/{city}_best_arima.pkl"
        pickle.dump(model_arima, open(path, "wb"))
    elif best_model_name == 'Prophet':
        path = f"results/{city}_best_prophet.pkl"
        pickle.dump(model_prophet, open(path, "wb"))
    elif best_model_name == 'XGBoost':
        path = f"results/{city}_best_xgb.pkl"
        joblib.dump(model_xgb, path)
    elif best_model_name == 'LSTM':
        path = f"results/{city}_best_lstm.h5"
        model_lstm.save(path)

    city_results.append({
        'City': city,
        'Best_Model': best_model_name,
        'RMSE': best_rmse,
        'Save_Path': os.path.abspath(path)
    })

# ==== Step 6. Save Summary ====
summary_df = pd.DataFrame(city_results)
summary_path = "results/city_model_summary.csv"
summary_df.to_csv(summary_path, index=False)
print("\n✅ Summary saved at:", os.path.abspath(summary_path))
print(summary_df)

Raw dataset shape: (29531, 16)
✅ Clean dataset saved at: /content/results/clean_city_data.csv

================= Ahmedabad =================


INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpnd7erjuf/ukj0_jor.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpnd7erjuf/mli0j6fg.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.12/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=81643', 'data', 'file=/tmp/tmpnd7erjuf/ukj0_jor.json', 'init=/tmp/tmpnd7erjuf/mli0j6fg.json', 'output', 'file=/tmp/tmpnd7erjuf/prophet_modelpuhy4q9n/prophet_model-20251023123210.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
12:32:10 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
12:32:10 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


Prophet failed for Ahmedabad: Dataframe has no rows.
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step


INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpnd7erjuf/qsq9tcwh.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpnd7erjuf/b4aoefhp.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.12/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=55876', 'data', 'file=/tmp/tmpnd7erjuf/qsq9tcwh.json', 'init=/tmp/tmpnd7erjuf/b4aoefhp.json', 'output', 'file=/tmp/tmpnd7erjuf/prophet_model1oxamcdb/prophet_model-20251023123222.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
12:32:22 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing



📊 Ahmedabad results:
                MAE       RMSE
ARIMA    17.068461  23.311652
XGBoost  14.217481  20.116114
LSTM     16.421110  21.026806
🏆 Best model for Ahmedabad: XGBoost (RMSE=20.12)

================= Aizawl =================


...

Prophet failed for Talcher: Dataframe has no rows.
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step

📊 Talcher results:
                MAE        RMSE
ARIMA    99.439935  107.129942
XGBoost  17.676828   30.628640
LSTM     26.824665   45.072418
🏆 Best model for Talcher: XGBoost (RMSE=30.63)

================= Thiruvananthapuram =================


INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpnd7erjuf/l_adg5cl.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpnd7erjuf/mwszostx.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.12/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=56154', 'data', 'file=/tmp/tmpnd7erjuf/l_adg5cl.json', 'init=/tmp/tmpnd7erjuf/mwszostx.json', 'output', 'file=/tmp/tmpnd7erjuf/prophet_model750m5d6m/prophet_model-20251023123503.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
12:35:03 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
12:35:03 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


Prophet failed for Thiruvananthapuram: Dataframe has no rows.
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step

📊 Thiruvananthapuram results:
               MAE       RMSE
ARIMA    9.455768  12.747514
XGBoost  6.044753   8.761222
LSTM     7.369290  10.727797
🏆 Best model for Thiruvananthapuram: XGBoost (RMSE=8.76)

================= Visakhapatnam =================


INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpnd7erjuf/ohz9mpl6.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpnd7erjuf/_jejtv8j.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.12/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=25471', 'data', 'file=/tmp/tmpnd7erjuf/ohz9mpl6.json', 'init=/tmp/tmpnd7erjuf/_jejtv8j.json', 'output', 'file=/tmp/tmpnd7erjuf/prophet_modelaml0mu83/prophet_model-20251023123507.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
12:35:07 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
12:35:07 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


Prophet failed for Visakhapatnam: Dataframe has no rows.
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step  

📊 Visakhapatnam results:
                MAE       RMSE
ARIMA    19.264551  26.562763
XGBoost  12.309160  17.615974
LSTM     16.255266  22.392741
🏆 Best model for Visakhapatnam: XGBoost (RMSE=17.62)

✅ Summary saved at: /content/results/city_model_summary.csv
                  City Best_Model       RMSE  \
0            Ahmedabad    XGBoost  20.116114   
1               Aizawl       LSTM   2.816050   
2            Amaravati    XGBoost   8.542596   
3             Amritsar    XGBoost  20.910291   
4            Bengaluru    XGBoost   9.948036   
5               Bhopal    XGBoost   6.559585   
6         Brajrajnagar    XGBoost  29.381492   
7           Chandigarh      ARIMA   6.960819   
8              Chennai    XGBoost  18.637170   
9           Coimbatore    XGBoost   8.207308   
10               Delhi    XGBoost  43.327750   
11           Ernakulam       LSTM   5.279909   
12            Gurug